<a href="https://colab.research.google.com/github/jbarker6706/MSDS462Final/blob/master/Chap462FinalNotebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is the publisher that starts the whole process. I simulate a camera by loading images of hamburger and waffle turtlebot3s. Then I publish the images after converting to ROS format for publishing. The script is called image_sim_pub.py All of the code in this script requires a ROS environment to run.

In [0]:
#!/usr/bin/env python
import rospy

import sys
import cv2
import os

from cv_bridge import CvBridge
from sensor_msgs.msg import Image

def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        dim = (224, 224)
        # resize image
        resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)

        if img is not None:
            images.append(resized)
    return images

waffles = []
hamburgers = []

def start_node():
    rospy.init_node('image_sim_pub')
    rospy.loginfo('image_sim_pub node started')
    waffledir = "/home/jbarker6706/Documents/MSDS462/assignment8/train/tb3waffle/"
    waffles = load_images_from_folder(waffledir)
    hamburgerdir = "/home/jbarker6706/Documents/MSDS462/assignment8/train/tb3hamburger/"
    hamburgers = load_images_from_folder(hamburgerdir)
    icnt = 0
    bridge = CvBridge()
    pub = rospy.Publisher('image', Image, queue_size=10)
    while not rospy.is_shutdown():
        imgMsg = bridge.cv2_to_imgmsg(waffles[icnt%len(waffles)], "bgr8")
        if(imgMsg != None):     
            pub.publish(imgMsg)
        rospy.sleep(10)  # 1 Hz
        imgMsg = bridge.cv2_to_imgmsg(hamburgers[icnt%len(hamburgers)], "bgr8")    
        if(imgMsg != None):     
            pub.publish(imgMsg)
        rospy.sleep(10)  # 1 Hz
        ++icnt

if __name__ == '__main__':
    try:
        start_node()
    except rospy.ROSInterruptException:
        pass


This is a subscriber and publisher. Initially this script was going to use a service to classify the image it received from image_sim_pub.py. I have a ImageClassify service but there is an issue with the fine-tuned VGG16 model that functioned in the confines of Jupyter. It failed to work within the framework of ROS. So I faked the classification to allow the process to continue. This script retreive_image.py gets the TB3 image from the image_sim_pub node and has the image classified. The image classification is published.

In [0]:
#!/usr/bin/env python
import rospy
from sensor_msgs.msg import Image
from std_msgs.msg import String

from cv_bridge import CvBridge
import cv2

rospy.init_node('tb3_fetch_pub')
rospy.loginfo('tb3_fetch_pub started')
pub = rospy.Publisher('tb3type', String, queue_size=10)
tbtype=0
tb3Msg = "burger"

def process_image(msg):
    try:
        global tbtype
        # convert sensor_msgs/Image to OpenCV Image
        bridge = CvBridge()
        orig = bridge.imgmsg_to_cv2(msg, "bgr8")
        drawImg = orig
        # show results
        rospy.loginfo('process_image type = ' + str(tbtype))
        showImage(drawImg)
        if(tbtype%2==0):
            tb3Msg = "burger"
        else:
            tb3Msg = "waffle"
        pub.publish(tb3Msg)
        tbtype += 1
    except Exception as err:
        print err

def showImage(img):
    cv2.imshow('image', img)
    cv2.waitKey(1)

def start_node():
#    rospy.init_node('retrieve_image')
    rospy.loginfo('retrieve_image part of tb3_fetch_pub node started')
    rospy.Subscriber("image", Image, process_image)
    rospy.spin()

if __name__ == '__main__':
    try:
        start_node()
    except rospy.ROSInterruptException:
        pass

The following two scripts are the two attempts to use a VGG16 fine-tuned model to classify burger or waffle. The two after are the files used to create the ImageClassify service. The classification failed because ROS is a framework which uses Python 2.7. Next time I will use ROS2. I am able to load the fine-tuned VGG16 trained model and print out a summary of the DNN. When I attempt to use the predict function the method fails. This is because the keras model has deprecated some of the features required in python 2.7. In particular it states that the float type is not recognized by the graph. While the same code works in Jupyter with 3.6. Here are retreive_image2.py and retreive_image_test.py.

In [0]:
#!/usr/bin/env python
import rospy
from sensor_msgs.msg import Image
from cv_bridge import CvBridge
import cv2

from std_msgs.msg import String
import tensorflow as tf

import numpy as np
from keras.models import load_model
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import decode_predictions

#from image_classify_model import ImageClassifyModel

config = tf.ConfigProto(
device_count={'GPU': 1},
intra_op_parallelism_threads=1,
allow_soft_placement=True
)

config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.6

session = tf.Session(config=config)
tf.keras.backend.set_session(session)

# load model
model = load_model('vgg16_tb3_burger_waffle.h5')
model._make_predict_function()
model.summary()

image_class = ["tb3hamburger", "tb3waffle"]

def classify_image(image):
    print("first")
    print(type(image))
    bridge = CvBridge()
    orig = bridge.imgmsg_to_cv2(image, "bgr8")

    image_array = img_to_array(orig)
    print("second")
    image_array = image_array.reshape((1, image_array.shape[0], 
                  image_array.shape[1], image_array.shape[2]))
    print("third")
    image_array = preprocess_input(image_array)
    print("fourth")
    yhat = model.predict(image_array)
    print("last")
    print(yhat)
    return image_class[np.argmax(yhat)]

def process_image(msg):
    try:
        # convert sensor_msgs/Image to OpenCV Image
        #bridge = CvBridge()
        #orig = bridge.imgmsg_to_cv2(msg, "bgr8")
        #drawImg = orig
        # show results
        #showImage(drawImg)
        print("First")
        classmsg = classify_image(msg)
        print("Last")
        print classmsg
    except Exception as err:
        print err

def showImage(img):
    cv2.imshow('image', img)
    cv2.waitKey(1)

def start_node():
    rospy.init_node('retrieve_image')
    rospy.loginfo('retrieve_image node started')
    rospy.Subscriber("image", Image, process_image)
    rospy.spin()

if __name__ == '__main__':
    try:
        start_node()
    except rospy.ROSInterruptException:
        pass

In [0]:
#!/usr/bin/env python
import rospy

from image_sim.srv import ImageClassify

import sys

from sensor_msgs.msg import Image
from cv_bridge import CvBridge
import cv2


def process_image(msg):
    try:
       
        rospy.wait_for_service('image_classify')

        image_classifier = rospy.ServiceProxy('image_classify', ImageClassify)

        category = str(image_classifier(msg))

        print category

        # convert sensor_msgs/Image to OpenCV Image
        bridge = CvBridge()
        orig = bridge.imgmsg_to_cv2(msg, "bgr8")
        drawImg = orig
        # show results
        showImage(drawImg)
    except Exception as err:
        print err

def showImage(img):
    cv2.imshow('image', img)
    cv2.waitKey(1)

def start_node():
    rospy.init_node('retrieve_image')
    rospy.loginfo('retrieve_image node started')
    rospy.Subscriber("image", Image, process_image)
    rospy.spin()

if __name__ == '__main__':
    try:
        start_node()
    except rospy.ROSInterruptException:
        pass

Here are the two files used to create the ImageClassify service: image_classify_model.py and image_classifier_service.py. As you can see some of the code is commented out from attempts to figure out the problem. I got the service to the point where I got the same graph failure. I thought it was a problem with my service until I punted and tried to classify without a service. I then realized it was a keras issue with ROS and python 2.7.

In [0]:
#!/usr/bin/python

import rospy
from std_msgs.msg import String
from sensor_msgs.msg import Image
from cv_bridge import CvBridge
import cv2

import numpy as np
from keras.models import load_model
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import decode_predictions


class ImageClassifyModel():

    def __init__(self):
        self.model = load_model('vgg16_tb3_burger_waffle.h5')
        self.model,summary()
        self.image_class = ["tb3hamburger", "tb3waffle"]

 
    def classifyImage(image):
        orig = bridge.imgmsg_to_cv2(image, "bgr8")
        image_array = img_to_array(orig)
        image_array = image_array.reshape((1, image_array.shape[0], 
                      image_array.shape[1], image_array.shape[2]))
        image_array = preprocess_input(image_array)
        yhat = model.predict(image_array)
        msg = self.image_class[np.argmax(yhat)]
        return String(msg)


In [0]:
#!/usr/bin/env python

import rospy

#from image_classify_model import ImageClassifyModel

from image_sim.srv import ImageClassify,ImageClassifyResponse
from image_sim.srv import ImageClassifyRequest

from std_msgs.msg import String
from sensor_msgs.msg import Image
from cv_bridge import CvBridge
import cv2

import numpy as np
from keras.models import load_model
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import decode_predictions

model = load_model('vgg16_tb3_burger_waffle.h5')
model.summary()
image_class = ["tb3hamburger", "tb3waffle"]
bridge = CvBridge()

def classify(request):
    orig = bridge.imgmsg_to_cv2(request.imgmsg, "bgr8")
    image_array = img_to_array(orig)
    image_array = image_array.reshape((1, image_array.shape[0], 
                  image_array.shape[1], image_array.shape[2]))
    image_array = preprocess_input(image_array)
    drawImg = orig
    # show results
    showImage(drawImg)

#    yhat = model.predict(image_array)
#    msg = self.image_class[np.argmax(yhat)]
    msg = "hamburger"
    return String(msg)
#    my_classifier = ImageClassifyModel        
#    return ImageClassifyResponse(my_classifier.classifyImage(request))

def showImage(img):
    cv2.imshow('image', img)
    cv2.waitKey(1)

rospy.init_node('image_classifier_service')
service = rospy.Service('image_classify', ImageClassify, classify)

rospy.spin()

Here is the file that has the turtlebot3 simulate package retrieval based on the published classification. I use turtlebot3_emptyworld to simplify navigation. Essentially I simulate the following steps 
1. Go to the loaction of the package 
2. Approximate a 90 degree rotation 
3. Pickup hamburger or waffle
4. Return to dock
I had planned on creating a composite architecture which uses primitive and composite components with an execute function to do the work in each step. A primitive component would be one of the items above and a composite would contain a list of the primitives. The composites execute function traverses the list and calls execute for each compnent in the list.This is a well known design pattern great for building step by step processes. It makes changing processes very simple. Create new primitives or add different primitives to composite list. Alas the lost time with the service has pushed this into a future project. The file is called tb3fetch_subscriber.py.

In [0]:
#!/usr/bin/env python


import rospy
from std_msgs.msg import String

from geometry_msgs.msg import Twist, Vector3
from math import pi


pub = rospy.Publisher('/cmd_vel', Twist, queue_size=1)
move = Twist()

def burgergo():
    move.linear.x = 0.2
    move.linear.y = 0.0
    duration = 5
    pub.publish(move)
    rospy.sleep(duration)
    move.linear.x = 0.0
    move.linear.y = 0.0
    pub.publish(move)

def wafflego():
    move.linear.x = 0.2
    move.linear.y = 0.0
    duration = 7
    pub.publish(move)
    rospy.sleep(duration)
    move.linear.x = 0.0
    move.linear.y = 0.0
    pub.publish(move)


def rotate90():
    move.linear.x = 0.0
    move.linear.y = 0.0
    duration = 10
    move.angular.z = pi*2/4/duration
    pub.publish(move)
    rospy.sleep(duration)
    move.angular.z = 0
    pub.publish(move)

def callback(msg):
    print msg.data
    if(msg.data == "burger"):
        burgergo()
        rotate90()
        print "get " + msg.data
        rotate90()
        burgergo()
        rotate90()
        rotate90()
    else:
        wafflego()
        rotate90()
        print "get " + msg.data
        rotate90()
        wafflego()
        rotate90()
        rotate90()


rospy.init_node('tb3fetch_subscriber')

sub = rospy.Subscriber('tb3type', String, callback)

rospy.spin()